In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)
data=pd.concat([train,test])
data["Stay_In_Current_City_Years"].replace("4+","4",inplace=True)
data["Age"].replace('0-17','17',inplace=True)
data["Age"].replace('55+','56',inplace=True)
data["Age"].replace('26-35','30',inplace=True)
data["Age"].replace('46-50','48',inplace=True)
data["Age"].replace('51-55','53',inplace=True)
data["Age"].replace('36-45','40',inplace=True)
data["Age"].replace('18-25','21',inplace=True)
data["Gender"].replace("F",0,inplace=True)
data["Gender"].replace('M',1,inplace=True)
data["City_Category"].replace('A',3,inplace=True)
data["City_Category"].replace('B',2,inplace=True)
data['City_Category'].replace('C',1,inplace=True)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data["Product_ID"]=le.fit_transform(data["Product_ID"])
train_new=data.iloc[:550068,:]
test_new=data.iloc[550068:,:]
test_new.drop('Purchase',inplace=True,axis=1)
train_new["Age"]=pd.to_numeric(train_new["Age"])
train_new["Stay_In_Current_City_Years"]=pd.to_numeric(train_new["Stay_In_Current_City_Years"])
x=train_new.drop('Purchase',axis=1)
y=train_new["Purchase"]
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(x,y, test_size =0.2)

C:\Users\Nishant Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\Nishant Kumar\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Nishant Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\U

In [4]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)
pred_lr=lr.predict(x_cv)
np.sqrt(mean_squared_error(pred_lr,y_cv))

4609.778750152604

In [21]:
import time
import winsound
from xgboost import XGBRegressor
import xgboost as xgb
regr = xgb.XGBRegressor(colsample_bytree=0.2,
                      gamma=0.0,
                      learning_rate=0.1,
                      max_depth=6,
                      min_child_weight=0.5,
                      n_estimators=7200,
                      reg_alpha=0.9,
                      reg_lambda=0.6,
                      subsample=0.2,
                      seed=42,
                      silent=1)
s=time.time()
regr.fit(x_train,y_train)
print("Fitting done")
answers=regr.predict(x_cv)
print("Rmse value",np.sqrt(mean_squared_error(answers,y_cv)))
print("Time taken:",time.time()-s)
winsound.MessageBeep()

Fitting done
Rmse value 2517.7388770823677
Time taken: 3635.3727400302887


In [22]:
test_new["Age"]=pd.to_numeric(test_new["Age"])
test_new["Stay_In_Current_City_Years"]=pd.to_numeric(test_new["Stay_In_Current_City_Years"])
s=time.time()
solution=regr.predict(test_new)
print("Time taken:",time.time()-s)

C:\Users\Nishant Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Nishant Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Time taken: 359.80927896499634


In [23]:
submission=pd.read_csv("Sample_Submission.csv")

In [24]:
submission.head()

,User_ID,Product_ID,Purchase
0,1000004,P00128942,14776.5160
1,1000009,P00113442,11317.6390
2,1000010,P00288442,7024.1390
3,1000010,P00145342,3587.3190
4,1000011,P00053842,1817.5469


In [25]:
test.head(5)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,0.0
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,0.0
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,0.0
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,0.0
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [26]:
submission["User_ID"]=test["User_ID"]
submission["Product_ID"]=test["Product_ID"]
submission["Purchase"]=solution

In [28]:
print("Rmse value",np.sqrt(mean_squared_error(answers,y_cv)))



Rmse value 2517.7388770823677
